In [7]:
#!pip3 install undetected-chromedriver
#!pip3 install undetected-chromedriver selenium pandas
#!pip3 install openpyxl


In [4]:
import time
import datetime
import pandas as pd
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [8]:


# ---- Function to Scrape LinkedIn Jobs Using Selenium ----
def scrape_linkedin_jobs(keyword, location):
    print("\n🔍 Scraping LinkedIn Jobs...\n")

    # Configure Selenium WebDriver (Headless Mode)
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run without opening a browser
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = uc.Chrome(options=options)
    
    # Generate LinkedIn job search URL
    search_url = f"https://www.linkedin.com/jobs/search?keywords={keyword.replace(' ', '%20')}&location={location.replace(' ', '%20')}"
    driver.get(search_url)
    
    # ✅ Scroll to load more jobs
    for _ in range(3):  
        driver.execute_script("window.scrollBy(0, 800);")
        time.sleep(2)

    # ✅ Wait for job listings to appear
    wait = WebDriverWait(driver, 15)  # Increased wait time
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "base-card")))

    jobs = []

    # ✅ Find all job listings
    job_elements = driver.find_elements(By.CLASS_NAME, "base-card")  

    for job in job_elements[:10]:  # Limit to top 10 jobs
        try:
            title_element = job.find_element(By.CSS_SELECTOR, "h3")  # Updated selector for job title
            title = title_element.text.strip()

            company_element = job.find_element(By.CSS_SELECTOR, "h4")  # Updated selector for company name
            company = company_element.text.strip()

            link = job.find_element(By.TAG_NAME, "a").get_attribute("href")

            jobs.append({"title": title, "company": company, "link": link, "source": "LinkedIn"})
        except Exception as e:
            print(f"⚠️ Skipping a job entry due to error: {e}")
            continue

    driver.quit()
    return jobs

# ---- Run the Script and Save to Excel ----
if __name__ == "__main__":
    keyword = input("Enter job title (e.g., Software Engineer): ")
    location = input("Enter location (e.g., Remote, New York, Berlin): ")

    linkedin_jobs = scrape_linkedin_jobs(keyword, location)

    if linkedin_jobs:
        df = pd.DataFrame(linkedin_jobs)

        # ✅ Save to Excel
        today_date = datetime.date.today().strftime("%Y-%m-%d")
        filename = f"linkedin_jobs_{today_date}.xlsx"
        df.to_excel(filename, index=False)
        
        print(f"\n✅ Jobs saved to {filename}")
    else:
        print("\n❌ No LinkedIn jobs found.")



🔍 Scraping LinkedIn Jobs...


✅ Jobs saved to linkedin_jobs_2025-02-23.xlsx
